In [15]:
#DLSProcessing
#Adapting old DLS python scripts for UTSA data and RapfLab generally
#Rebecca Rapf
#June 24, 2021


import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import glob
import os
import pandas as pd
#import datetime
import DLSFunctionLibrary as DFL


Path0 = '/Users/rebeccarafp/Documents/RapfLabPython/DLS/210617/CorrFunctions'
#SAVE_DATA = 'yes'
SAVE_DATA = 'yes'
FileType = "csv"


os.chdir(Path0)
print(os.getcwd())
outputfolder = "Fitted"
isdir = os.path.isdir(outputfolder)
if isdir == False:
    os.mkdir(outputfolder)
    
#ListFiles = pd.read_csv('PlateReaderDLSCorrFilesedit0512.csv')
ListFiles = sorted(glob.glob("*"+FileType))
ListFiles.sort(key=os.path.getmtime)
print(ListFiles)

for entry in range(len(ListFiles)):
    p_file = ListFiles[entry]
    print(p_file)
    prename = p_file[:-(len(FileType)+1)]
    #print(prename)
    
    
    
    #datacolumns = ["t_us", "C_1", "C_2", "C_3", "C_4","C_5"]
    data = pd.read_csv(p_file, encoding= 'unicode_escape', skiprows = 1)
    
    num_meas = int(len(data.columns)/2)
    #print(num_meas)
    
    #for i in num_meas
    
    #data = read_in.copy()
    #print(data.head())
    #data = data.drop(data.index[0])
    if num_meas == 5:
        data = data.drop(data.columns[[2,4,6,8]], axis = 1)#,inplace = True)
        data = data.rename(columns={data.columns[0]: "t_us",data.columns[1]: "C_1",
                                data.columns[2]: "C_2", data.columns[3]: "C_3",
                                data.columns[4]: "C_4",data.columns[5]: "C_5"})
    elif num_meas == 3:
        data = data.drop(data.columns[[2,4]], axis = 1)#,inplace = True)
        data = data.rename(columns={data.columns[0]: "t_us",data.columns[1]: "C_1",
                                data.columns[2]: "C_2", data.columns[3]: "C_3"})
    else: 
        print("Check the number of measurements")
    #print(data.head())
    
    #print(num_meas)


    modelParamsDict = DFL.initializeExponentialModelParams(298);
    #print(modelParamsDict)

    FitArray = pd.DataFrame()
    R2Array = np.zeros(num_meas)
    RadiiArray = np.zeros(num_meas)
    RadVarArray = np.zeros(num_meas)
    RadStdArray = np.zeros(num_meas)
    
    #IndexNames = range(num_meas-2)#+["AvgNormCorr:","AvgUnNormCorr"]
    #print(IndexNames)
    
    
    CleanedData = pd.DataFrame()    
    SumNormCorr = np.zeros(len(data))
    SumUnNormCorr = np.zeros(len(data))
    SumNormAvg = 0
    
    for entry in range(num_meas):
        if entry == 0:
            CleanedData[prename+"DecayTime"] = data.iloc[:,0]/1000
            #print(CleanedData)
        
        CopyArray =  data.iloc[:,entry+1]
        #print(CopyArray)
        #CopyArray = CopyArray.sub(1)
        #print(CopyArray)
        NormAvg = CopyArray[1:25].mean()
        #print(NormAvg)
        NormCopyArray = CopyArray.div(NormAvg)
        CleanedData[prename+"Corr"+str(entry)] = NormCopyArray
        
        SumNormAvg += NormAvg
        
        SumUnNormCorr += CopyArray
        SumNormCorr += NormCopyArray  
        
    AvgNormCorr = SumNormCorr/num_meas
    AvgUnNormCorr = SumUnNormCorr/SumNormAvg
    
    CleanedData[prename+"AvgNormCorr"] = AvgNormCorr
    CleanedData[prename+"AvgUnNormCorr"] = AvgUnNormCorr
    
    CleanedData.to_csv("Norm/"+prename+'CleanCorr.csv')
    
    for entry in range(num_meas):
        #print(entry)
        Decays = CleanedData.iloc[:,0]#/1000 #units needs to be in ms. Plate Reader Data is in us.
        #print(Decays)
        
        Corr = CleanedData.iloc[:,entry+1]
        #print(Corr)
        CurrName = CleanedData.columns[entry+1]
    
        modelParamsDict["DecayTime"] = Decays
        #print AvgData.iloc[:,i]
        Radius, RadVar = curve_fit(DFL.ExponentialModel, modelParamsDict, Corr, maxfev = 100000)
        #print(Radius)
        RadiiArray[entry] = Radius
        RadVarArray[entry] = RadVar
        RadStdArray[entry] = np.sqrt(RadVar)
        #print(RadStdArray)
    
        FitArray['Fit'+CurrName] = (DFL.ExponentialModel(modelParamsDict,Radius))
    #    
        R0 = np.sum((DFL.ExponentialModel(modelParamsDict,Radius) - Corr)**2)
    #    #print AvgR0
        R1 = np.sum((Corr - np.mean(Corr))**2)
        R2 = 1 - (R0/R1)
        R2Array[entry] = R2
        
        #print(R2)
    
    ExtractedRadii = pd.DataFrame()#index = IndexNames)
    
    ExtractedRadii[prename+'Radii']=RadiiArray
    ExtractedRadii[prename+'RadVar']=RadVarArray
    ExtractedRadii[prename+'RadStd']=RadStdArray
    ExtractedRadii[prename+'R2']=R2Array
    #
    
    if SAVE_DATA == 'yes':
        FitArray.to_csv("./"+outputfolder+"/"+prename+'Fits.csv')
        ExtractedRadii.to_csv("./"+outputfolder+"/"+prename+'Radii.csv')
    else:
        print("DATA WAS NOT SAVED")

/Users/rebeccarafp/Documents/RapfLabPython/DLS/210617/CorrFunctions
['210616OOApost.csv', '210612OICApost.csv', '210610OVApost.csv', '210615OBApost.csv', '210610OVApost_2.csv', '210616OOApre.csv', '210612OICApre.csv', '210610OVApre.csv', '210615OBApre.csv']
210616OOApost.csv
210612OICApost.csv
210610OVApost.csv
210615OBApost.csv
210610OVApost_2.csv
210616OOApre.csv
210612OICApre.csv
210610OVApre.csv
210615OBApre.csv
